Classification Prob.

10 input var
i binary optput var (benign or malignant)

569 data samples
samples[:100] -> test set
samples[100:200] -> validation set
samples[200:] -> training set

!! train_test_split() -> ban !! 
// use numpy slicing

if ? in b-c-w.data:
    remove row

data[:,0] -> remove

normalize

set output (malignant=1, benign=0)

train, test, validation set

<model>
9 inputs
10 hn (ReLu)
1 optn (Sigmoid)

In [ ]:
import tensorflow as tf
import numpy as np

# generating examples (samples)
x = np.array([0, 1, 2, 3, 4])
y = x*2 + 1

# generating neural network modeling

model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Dense(1, input_shape=(1,)))
model.compile('SGD', 'mse', metrics=['accuracy'])

# # before learning
# print('Before Learning:')
# print('Targets: ', y[2:])
# print('Predictions: ', model.predict(x[2:]).flatten())
# print(model.get_weights())

# after training
model.fit(x_input, y_input, epochs=epo, verbose=0)
print('After Learning:')
print('Targets: ', y[2:])
print('Predictions: ', model.predict(x[2:]).flatten())
print(model.get_weights())